In [1]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
# 88 percent accuracy
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1, 2" # Will use only 2nd, 3rd and 4th gpu


import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [ ]:

# load ascii text and covert to lowercase
filename = "eminescu.csv"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

('Total Characters: ', 180037)
('Total Vocab: ', 55)


In [ ]:

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

#Get a weight
filename = "weights-improvement-27-1.6371-eminescu.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-eminescu.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, nb_epoch=1050, batch_size=64, callbacks=callbacks_list)

('Total Patterns: ', 179937)
Epoch 1/1050
179937/179937 [==============================] - 941s - loss: 1.6246   
Epoch 2/1050
179937/179937 [==============================] - 902s - loss: 1.6317   
Epoch 3/1050
179937/179937 [==============================] - 904s - loss: 1.6314   
Epoch 4/1050
179937/179937 [==============================] - 890s - loss: 1.6323   
Epoch 5/1050
179937/179937 [==============================] - 914s - loss: 1.6273   
Epoch 6/1050
179937/179937 [==============================] - 909s - loss: 1.6255   
Epoch 7/1050
179937/179937 [==============================] - 929s - loss: 1.6217   
Epoch 8/1050
179937/179937 [==============================] - 915s - loss: 1.6227   
Epoch 9/1050
179937/179937 [==============================] - 911s - loss: 1.6208   
Epoch 10/1050
179937/179937 [==============================] - 890s - loss: 1.6169   
Epoch 11/1050
179937/179937 [==============================] - 915s - loss: 1.6152   
Epoch 12/1050
179937/179937 [=====

KeyboardInterrupt: 